1. In this final lab, we will model our data. Import sklearn `train_test_split` and separate the data.

2. Separate X_train and X_test into numerical and categorical (X_train_cat , X_train_num , X_test_cat , X_test_num)

3. Use X_train_num to fit scalers.  Transform BOTH X_train_num and X_test_num.

4. Encode the categorical variables X_train_cat and X_test_cat (See the hint below for encoding categorical data!!!)

5. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

6. Try a simple linear regression with all the data to see whether we are getting good results.

7. Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

8. Use the function to check `LinearRegressor` and `KNeighborsRegressor`.

9. You can check also the `MLPRegressor` for this task!

10. Check and discuss the results.  



You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):
Encoder Type | Column 
-----------------|-----------------
One hot | state
Ordinal | coverage
Ordinal | employmentstatus
Ordinal | location code
One hot | marital status
One hot | policy type
One hot | policy
One hot | renew offercustomer_df
One hot | sales channel
One hot | vehicle class
Ordinal | vehicle size
Ordinal | education
One hot | response
One hot | gender


### Dummy code

data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})

In [1]:
import pandas as pd #tablas https://pandas.pydata.org/docs/user_guide/pyarrow.html
import numpy as np #estadistica https://numpy.org/doc/1.25/user/index.html#user
import matplotlib.pyplot as plt #gráficos https://matplotlib.org/stable/plot_types/index.html
import seaborn as sns #gráficos https://seaborn.pydata.org/tutorial/regression.html
from sklearn import linear_model #modelo de regresion https://scikit-learn.org/stable/
from sklearn.preprocessing import MinMaxScaler # do not use the function Normalise() - it does something entirely different
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from datetime import datetime #https://docs.python.org/3/library/datetime.html
from datetime import timedelta #this is a good time to learn about dates and timestamps. You can find some info here:
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from scipy.stats import ttest_1samp
import scipy.stats as st
from sklearn.neighbors import KNeighborsClassifier
from mlxtend.plotting import plot_decision_regions

ModuleNotFoundError: No module named 'mlxtend'

1. In this final lab, we will model our data. Import sklearn `train_test_split` and separate the data.

In [ ]:
df_cat = pd.read_csv('categorical22_08.csv')
df_num = pd.read_csv('numerical.csv')

In [ ]:
df_num

In [ ]:
df_cat

In [ ]:
df_cat = df_cat.astype(str)
df_cat

In [ ]:
df= pd.concat([df_num, df_cat], axis=1)
df = df.drop(columns=["effective_to_date"])
df


In [ ]:
df.to_csv("23_08_df.csv", index=False) 

2. Separate X_train and X_test into numerical and categorical (X_train_cat , X_train_num , X_test_cat , X_test_num)

In [ ]:
X = df.drop(['total_claim_amount'],axis=1)
y = df['total_claim_amount']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

3. Use X_train_num to fit scalers.  Transform BOTH X_train_num and X_test_num.

In [ ]:
X_train_num = X_train.select_dtypes(include = np.number)
X_test_num  = X_test.select_dtypes(include = np.number)
X_train_cat = X_train.select_dtypes(include = object)
X_test_cat  = X_test.select_dtypes(include = object)

In [ ]:
X_train_num

In [ ]:
X_test_cat

In [ ]:
MinMaxtransformer = MinMaxScaler().fit(X_train_num) #IMPORTANTE ESTO ES EL NOMBRE DEL TRANSFORMADOR
transformer = MinMaxScaler().fit(X_train_num) #Aplicar el min y el max, pero recuerda que X  tiene que ser valores numericos
x_train_norm= transformer.transform(X_train_num) #Convertirlo entre 0 y 1 
print(x_train_norm.shape)
x_train_norm
x_train_normalized= pd.DataFrame(x_train_norm, columns=X_train_num.columns) #df.normalizado para asi poder tener un data set normalizado al que llamar
x_train_normalized

In [ ]:
x_test_norm= transformer.transform(X_test_num) #Convertirlo entre 0 y 1 
print(x_test_norm.shape)
x_test_norm
x_test_normalized= pd.DataFrame(x_test_norm, columns=X_test_num.columns) #df.normalizado para asi poder tener un data set normalizado al que llamar
x_test_normalized

4. Encode the categorical variables X_train_cat and X_test_cat (See the hint below for encoding categorical data!!!)

One encoding everything :)

In [ ]:
X_train_cat = X_train.select_dtypes(include = object)
X_test_cat  = X_test.select_dtypes(include = object)

In [ ]:

encoder = OneHotEncoder(handle_unknown='error', drop='first').fit(X_train_cat )
categoricals_train_encoded = encoder.transform(X_train_cat).toarray()
categoricals_test_encoded = encoder.transform(X_test_cat ).toarray()

In [ ]:
categorical_train_encoded_df = pd.DataFrame(categoricals_train_encoded)
categorical_train_encoded_df

In [ ]:
categorical_test_encoded_df = pd.DataFrame(categoricals_test_encoded)
categorical_test_encoded_df

5. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

In [ ]:
X_train_norm_enc = pd.concat([x_train_normalized, categorical_train_encoded_df], axis=1)
X_train_norm_enc

In [ ]:
X_train_norm_enc.columns = X_train_norm_enc.columns.astype(str)

In [ ]:
X_test_norm_enc = pd.concat([x_test_normalized, categorical_test_encoded_df], axis=1)
X_test_norm_enc

In [ ]:
X_test_norm_enc.columns = X_test_norm_enc.columns.astype(str)

In [ ]:
X_train_norm_enc.dtypes

In [ ]:
X_test_norm_enc.dtypes

In [ ]:
y

6. Try a simple linear regression with all the data to see whether we are getting good results.

In [ ]:
lm = linear_model.LinearRegression()
lm.fit(X_train_norm_enc,y_train)

In [ ]:
predictions = lm.predict(X_test_norm_enc)
r2_score(y_test, predictions)

7. Great! Now define a function that takes a list of models and train (and tests) them so we can try a lot of them without repeating code.

In [ ]:
def train_and_test_regression_models(models, X_train, X_test, y_train, y_test, test_size=0.2, random_state=100):
    results = {}

    for model in models:
        model_name = model.__class__.__name__
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        r2 = r2_score(y_test, y_pred)
        results[model_name] = r2

    return results

8. Use the function to check `LinearRegressor` and `KNeighborsRegressor`.

In [ ]:

X_train = X_train_norm_enc
X_test= X_test_norm_enc

models_to_try = [LinearRegression(), KNeighborsRegressor()]

results = train_and_test_regression_models(models_to_try, X_train, X_test, y_train, y_test)
print(results)



 9. You can check also the `MLPRegressor` for this task!

In [4]:
from sklearn.neural_network import MLPRegressor
models_to_try = [MLPRegressor()]

results = train_and_test_regression_models(models_to_try, X_train, X_test, y_train, y_test)
print(results)

NameError: name 'train_and_test_regression_models' is not defined

10. All of them have a similar explanatory power, besides the KNN (not K defined)